In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf 
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.applications import MobileNetV3Small
#from tensorflow.keras.applications.convnext import ConvNeXtTiny
#from tensorflow.keras.applications.efficientnet import EfficientNetB2, EfficientNetB3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
#from tensorflow.keras.applications.resnet_rs import ResNetRS50
#from tensorflow.keras.applications.resnet_v2 import ResNet50V2
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Add, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, Layer, MaxPool2D, ReLU, Resizing
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import L2
from tqdm import tqdm

In [ ]:
# paths
proj_path = 'Documents'
dataset_path = proj_path + '/SCUT-FBP'
image_path = dataset_path + '/Cleaned_Images/'
rate_path = dataset_path + '/Rating_Collection' 

In [ ]:
import os
folder_path = image_path # replace with the actual folder path

for filename in os.listdir(folder_path):
    if "(1)" in filename:
        file_path = os.path.join(folder_path, filename)
        os.remove(file_path)

In [ ]:
print(tf.__version__)
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

2.10.0
Num GPUs Available 1


In [ ]:
# load data: y_arr = [rating, race, gender]
def load_data(img_dir, label_dir):
  all_ratings = pd.read_csv(label_dir, sep = ' ', header = None)
  all_ratings.columns = ['img_path', 'rating']

  img_arr = np.zeros([len(all_ratings), 224, 224, 3])
  y_arr = np.zeros([len(all_ratings), 3])

  for i in tqdm(range(len(all_ratings))):
  #for i in tqdm(range(500)):
    file_name = all_ratings.iloc[i, 0]
    race = file_name[0]
    gender = file_name[1]
    if race == 'A':
      y_arr[i, 1] = 0
    else:
      y_arr[i, 1] = 1

    if gender == 'M':
      y_arr[i, 2] = 0
    else:
      y_arr[i, 2] = 1 
        
    y_arr[i, 0] = all_ratings.iloc[i, 1]
    
    img = tf.io.read_file(img_dir + file_name)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.keras.layers.Resizing(224, 224)(img)
    img = preprocess_input(img)
    img_arr[i] = img

  return img_arr, y_arr

img_arr, y_arr = load_data(image_path, rate_path + '/All_labels.txt')

100%|█████████████████████████████████████████████████████████████████████████████| 5500/5500 [00:33<00:00, 164.74it/s]


In [ ]:
img_full_train, img_test, y_full_train, y_test = train_test_split(img_arr, y_arr, stratify = y_arr[:, 2], test_size = 0.2, 
                                                                  random_state = 0)
img_train, img_val, y_train, y_val = train_test_split(img_full_train, y_full_train, stratify = y_full_train[:, 2], 
                                                      test_size = 0.2, random_state = 0)

rating_train = y_train[:, 0]
rating_val = y_val[:, 0]
rating_test = y_test[:, 0]

gender_train = y_train[:, 1]
gender_val = y_val[:, 1]
gender_test = y_test[:, 1]

In [ ]:
class Input_Stream(Model):
    def __init__(self, num_maps, kernel_size, momentum_parameter, pool):
        super(Input_Stream, self).__init__()
        
        self.conv1 = Conv2D(num_maps, kernel_size, strides = 2, use_bias = False)
        self.bn1 = BatchNormalization(momentum = momentum_parameter)
        self.conv2 = Conv2D(num_maps, kernel_size, padding = 'same', use_bias = False)
        self.bn2 = BatchNormalization(momentum = momentum_parameter)
        self.conv3 = Conv2D(num_maps, kernel_size, padding = 'same', use_bias = False)
        self.bn3 = BatchNormalization(momentum = momentum_parameter)
        self.relu = ReLU()
        self.max_pool = MaxPool2D(pool_size = pool, strides = 2)
        
       
    
    def call(self, inputs):
        x = self.relu(self.bn1(self.conv1(inputs)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.max_pool(x)
        
        return x

In [ ]:
class Residual_Block(Model):
    def __init__(self, num_maps, kernel_size, momentum_parameter):
        super(Residual_Block, self).__init__()

        self.conv1 = Conv2D(num_maps, kernel_size, activation='relu', padding='same')
        self.bn1 = BatchNormalization(momentum = momentum_parameter)
        self.conv2 = Conv2D(num_maps, kernel_size, padding='same')
        self.add = Add()
        self.relu = ReLU()
        self.bn2 = BatchNormalization(momentum = momentum_parameter)
    
    def call(self, inputs):
        x = inputs
        fx = self.conv2(self.bn1(self.conv1(x)))
        concat = self.add([x, fx])
        out = self.bn2(self.relu(concat))
        return out

In [ ]:
class MT_ResNet(Model):
    def __init__(self):
        super(MT_ResNet, self).__init__()
        
        self.start_block = Input_Stream(num_maps = 64, kernel_size = 3, momentum_parameter = 0.5, pool = 3)
        self.resnet50 = ResNet50(include_top = False, pooling = None)
        self.resnet50_backbone = Model(self.resnet50.layers[7].input, self.resnet50.layers[-1].output)
        self.res_block1 = Residual_Block(num_maps = 2048, kernel_size = 1, momentum_parameter = 0.5)
        self.res_block2 = Residual_Block(num_maps = 2048, kernel_size = 1, momentum_parameter = 0.5)
        self.global_pool1 = GlobalAveragePooling2D()
        self.global_pool2 = GlobalAveragePooling2D()
        self.fc1 = Dense(1, name = 'fap', use_bias = False)
        self.fc2 = Dense(1, activation = 'sigmoid', name = 'gender', use_bias = False)
        
        
        
    def call(self, inputs):
        x = self.start_block(inputs)
        x = self.resnet50_backbone(x)
        x_fap = self.global_pool1(self.res_block1(x))
        x_fap = self.fc1(x_fap)
        x_gender = self.global_pool2(self.res_block2(x))
        x_gender = self.fc2(x_gender)
        
        return x_fap, x_gender

In [ ]:
class MT_ResNet(Model):
    def __init__(self):
        super(MT_ResNet, self).__init__()
        
        self.start_block = Input_Stream(num_maps = 64, kernel_size = 3, momentum_parameter = 0.5, pool = 3)
        self.resnet50 = ResNet50(include_top = False, pooling = 'avg')
        self.resnet50_backbone = Model(self.resnet50.layers[7].input, self.resnet50.layers[-1].output)
        
        self.fap_block = Sequential([
            Dense(128, activation='relu', use_bias=False),
            BatchNormalization(momentum=0.5),
            Dropout(0.3),
            Dense(64, activation='relu', use_bias=False),
            BatchNormalization(momentum=0.5),
            Dropout(0.3),
            Dense(1, name = 'fap', use_bias = False)
        ])

        self.gender_block = Sequential([
            Dense(8, activation='relu', use_bias=False),
            BatchNormalization(momentum=0.5),
            Dropout(0.3),
            Dense(1, activation = 'sigmoid', name = 'gender', use_bias = False)
        ])
        
        
    def call(self, inputs):
        x = self.start_block(inputs)
        x = self.resnet50_backbone(x)
        x_fap = self.fap_block(x)
        x_gender = self.gender_block(x)
        
        return x_fap, x_gender

In [ ]:
Loss_Functions = {'output_1': 'mse', 'output_2': 'binary_crossentropy'}
Loss_Weights = {'output_1': 2, 'output_2': 1}

In [ ]:
adam_optimizer = Adam(learning_rate = 0.001)
model = MT_ResNet()
model.compile(optimizer = adam_optimizer, loss = Loss_Functions, loss_weights = Loss_Weights)

In [ ]:
def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.01) 

    
    
required_callbacks = [
    #tf.keras.callbacks.ModelCheckpoint(filepath = 'checkpoint', monitor = 'val_loss', save_best_only = True),
    #tf.keras.callbacks.TensorBoard(),
    tf.keras.callbacks.EarlyStopping(patience = 30, restore_best_weights = True),
    tf.keras.callbacks.LearningRateScheduler(custom_scheduler),
    tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 10, verbose = 1, cooldown = 5, min_lr = 0.0000001),
    tf.keras.callbacks.TerminateOnNaN()
]

In [ ]:
model.fit(img_arr, [y_arr[:, 0], y_arr[:, 1]], batch_size = 4, epochs = 1000, verbose = 1, 
          callbacks = required_callbacks, validation_split = 0.2)

Epoch 1/1000
1100/1100 [==============================] - 95s 75ms/step - loss: 5.6255 - output_1_loss: 2.5031 - output_2_loss: 0.6193 - val_loss: 7.9925 - val_output_1_loss: 3.6561 - val_output_2_loss: 0.6804 - lr: 0.0010
Epoch 2/1000
1100/1100 [==============================] - 80s 72ms/step - loss: 2.3259 - output_1_loss: 0.8656 - output_2_loss: 0.5948 - val_loss: 1.5704 - val_output_1_loss: 0.4591 - val_output_2_loss: 0.6522 - lr: 0.0010
Epoch 3/1000
1100/1100 [==============================] - 80s 73ms/step - loss: 1.9553 - output_1_loss: 0.6804 - output_2_loss: 0.5944 - val_loss: 1.3458 - val_output_1_loss: 0.3743 - val_output_2_loss: 0.5972 - lr: 0.0010
Epoch 4/1000
1100/1100 [==============================] - 80s 73ms/step - loss: 1.8851 - output_1_loss: 0.6452 - output_2_loss: 0.5948 - val_loss: 681469.0625 - val_output_1_loss: 340702.7500 - val_output_2_loss: 63.6854 - lr: 0.0010
Epoch 5/1000
1100/1100 [==============================] - 80s 73ms/step - loss: 1.7709 - output_1

In [ ]:
model.save('Documents/SCUT-FBP/saved_model')

INFO:tensorflow:Assets written to: Documents/SCUT-FBP/saved_model\assets


INFO:tensorflow:Assets written to: Documents/SCUT-FBP/saved_model\assets
